In [1]:
import numpy as np
import cv2
import os
import json

In [2]:
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [3]:
train_data_path="/Users/Giles/Desktop/nicho/data/testData/train/"
test_data_path="/Users/Giles/Desktop/nicho/data/testData/test/"
valid_data_path="/Users/Giles/Desktop/nicho/data/testData/valid/"
base_data_path="/Users/Giles/Desktop/nicho/data/testData/"


In [46]:
path="/Users/Giles/Desktop/nicho/data/testData/train/"

#get list of subdirectories in path
folders=[f.path for f in os.scandir(path) if f.is_dir() ]

#initialize arrays to store image data and labels
data=[]
labels=[]
label_arr=[] #labels of folders
#iterate through each folder
for folder in folders:
    #get a list of all the images in the folder
    images=[img for img in os.listdir(folder) if img.endswith('.jpg') or img.endswith('.png')]

    #get label for current folder
    label=os.path.basename(folder)
    label_arr.append(label)
    #iterate through each image in the folder
    for image in images:

        #load the image
        img=cv2.imread(os.path.join(folder,image))

        #resize the images to uniform size       
        img=cv2.resize(img, (250,250),interpolation=cv2.INTER_LINEAR)

        ##viewing the images
        #cv2.imshow('pic', img)
        #cv2.waitKey(0)

        # Extract features from the image
        hist = cv2.calcHist([img], [0], None, [256], [0, 256])
        hist=hist/np.sum(hist)
        features = hist.flatten()

        # Optimize the features using XGBoost
        xgb_model = xgb.XGBRegressor()
        xgb_model.fit(features.reshape(-1, 1), np.zeros(features.shape[0]))

        #append the image data and label to the arrays
        data.append(features)
        labels.append(label)
        

#convert data and labels to numpy arrays
data=np.array(data)
labels=np.array(labels)



In [47]:
### trying to change the labels array to the foldernames that indicate type of cancer 
# and hence use the dictionary to change them to integers

for filename in os.listdir(path):
    file_path = os.path.join(path, filename)
    # make sure we're only working with files and not subdirectories
    if os.path.isfile(file_path):
        # open the file and read the label
        with open(file_path, 'r') as file:
            label = file.read()
        # write the folder name to the file
        with open(file_path, 'w') as file:
            file.write(folder_name)

In [48]:
#dictionary to respresent folder names as integers
label_index={}
for i in range(len(label_arr)):
    label_index[i]=label_arr[i]
    
print(label_index)


{0: 'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib', 1: 'Bengin cases', 2: 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa', 3: 'Malignant cases', 4: 'normal', 5: 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa'}


In [49]:
reverse_label_index = {v: k for k, v in label_index.items()}
print(reverse_label_index)

{'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib': 0, 'Bengin cases': 1, 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa': 2, 'Malignant cases': 3, 'normal': 4, 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa': 5}


In [52]:
print(labels[46])

adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib


In [51]:
##produces error due to non existent value in dictionary since its reading files in folder
## stil trying tosolve this
for i, string in enumerate(labels):
    labels[i] = label_index[string]

KeyError: 'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib'

In [41]:
#KNN
knn = KNeighborsClassifier()
knn.fit(data, labels)

KNeighborsClassifier()

In [ ]:
#CNN
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(1460,256,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(data, labels, epochs=10, batch_size=32, validation_split=0.2)